In [3]:
import ROOT

# Open the ROOT file (use the correct file extension, e.g., .root if it's a ROOT file)
file = ROOT.TFile("Data.root")

# List the contents of the file
file.ls()

# Assuming there is only one tree, or you know the tree name after listing the contents
tree_name = None

# Loop through the keys in the file to find a TTree object
for key in file.GetListOfKeys():
    obj = key.ReadObj()
    if isinstance(obj, ROOT.TTree):
        tree_name = obj.GetName()
        print(f"Found tree: {tree_name}")
        break

if tree_name:
    tree = file.Get(tree_name)

    # Print the branches of the tree to identify available branches
    print("Branches in the tree:")
    tree.Print()

    # Example: Print first 10 entries of a specific branch
    for i in range(10):
        tree.GetEntry(i)
        # Replace 'muon_pt' with the actual branch name after inspecting the tree
        # For now, let's just print out all the branches in the first entry
        print(f"Entry {i}:")
        for branch in tree.GetListOfBranches():
            branch_name = branch.GetName()
            value = getattr(tree, branch_name)
            print(f"  {branch_name}: {value}")
else:
    print("No TTree found in the ROOT file.")


Found tree: HASCO
Branches in the tree:
Entry 0:
  runNumber: 207490
  eventNumber: 17390906
  channelNumber: 207490
  mcWeight: 0.0
  pvxp_n: 19
  vxp_z: -2.9964637756347656
  scaleFactor_PILEUP: 0.0
  scaleFactor_ELE: 0.0
  scaleFactor_MUON: 0.0
  scaleFactor_BTAG: 0.0
  scaleFactor_TRIGGER: 0.0
  scaleFactor_JVFSF: 0.0
  scaleFactor_ZVERTEX: 0.0
  trigE: 0
  trigM: 1
  passGRL: 1
  hasGoodVertex: 1
  lep_n: 2
  lep_truthMatched: <cppyy.LowLevelView object at 0x7c65a427cdc0>
  lep_trigMatched: <cppyy.LowLevelView object at 0x7c65a427ce30>
  lep_pt: <cppyy.LowLevelView object at 0x7c65a427cdc0>
  lep_eta: <cppyy.LowLevelView object at 0x7c65a427de60>
  lep_phi: <cppyy.LowLevelView object at 0x7c65a427ded0>
  lep_E: <cppyy.LowLevelView object at 0x7c65a427cce0>
  lep_z0: <cppyy.LowLevelView object at 0x7c65a427cea0>
  lep_charge: <cppyy.LowLevelView object at 0x7c65a427db50>
  lep_type: <cppyy.LowLevelView object at 0x7c65a427dbc0>
  lep_flag: <cppyy.LowLevelView object at 0x7c65a427dc

In [4]:
import ROOT

# Open the ROOT file
mcFileName = "Data.root"  # Assuming you are in the correct directory
mcFile = ROOT.TFile.Open(mcFileName, "READ")
if not mcFile or mcFile.IsZombie():
    raise OSError(f"Failed to open file: {mcFileName}")

# Print the contents of the ROOT file to know the available histograms/trees
mcFile.ls()

# Get the TTree from the file
treeName = "HASCO"
tree = mcFile.Get(treeName)
if not tree:
    raise OSError(f"Failed to get TTree '{treeName}' from the file")

# Print the structure of the TTree
tree.Print()


TFile**		Data.root	
 TFile*		Data.root	
  KEY: TTree	HASCO;1	4-vectors + variables required for scaling factors
******************************************************************************
*Tree    :HASCO     : 4-vectors + variables required for scaling factors     *
*Entries :    62756 : Total =        18175558 bytes  File  Size =   10394987 *
*        :          : Tree compression factor =   1.75                       *
******************************************************************************
*Br    0 :runNumber : runNumber/I                                            *
*Entries :    62756 : Total  Size=     251602 bytes  File Size  =       1153 *
*Baskets :        1 : Basket Size=    1312768 bytes  Compression= 217.78     *
*............................................................................*
*Br    1 :eventNumber : eventNumber/I                                        *
*Entries :    62756 : Total  Size=     251612 bytes  File Size  =     251103 *
*Baskets :        1

In [5]:
import ROOT

# Function to read the tree and fill histograms
def readTreeToMllHistogram(tree, dataType):
    if dataType not in ["data", "MC"]:
        print(f"The data type should be \"data\" or \"MC\", not {dataType}")
        return None
    
    # Make the histogram to store the events
    # Use the dataType as a part of the name and title
    # Make 150 bins of size 1 GeV, from 50 GeV to 200 GeV
    # Note that units are in MeV, so 200 GeV is 200x10^3, or equivalently 200.e3

    mll = ROOT.TH1D(dataType, f"m_{{ll}}, {dataType}", 150, 50.e3, 200.e3)
    mll.Sumw2()

    # Run over all of the entries in the tree
    # In other words, run over all of the events

    for entryNum in range(tree.GetEntries()):
        tree.GetEntry(entryNum)

        # Ensure this is a di-lepton event
        # A di-lepton event will have a lep(ton) n(umber) of 2
        # If the lepton number is not 2, then start the loop again

        if getattr(tree, "lep_n") != 2:
            continue

        # We now know that there are two leptons
        # Get their kinematics
        # Note that these are arrays/lists of size lep_n

        pT = getattr(tree, "lep_pt")
        eta = getattr(tree, "lep_eta")
        phi = getattr(tree, "lep_phi")
        nrg = getattr(tree, "lep_E")

        # Convert the kinematics into a four-vector object
        # Recall that python is 0-indexed
        # That means that the first element in an array/list is 0, not 1

        lepton0 = ROOT.TLorentzVector()
        lepton0.SetPtEtaPhiE(pT[0], eta[0], phi[0], nrg[0])
        lepton1 = ROOT.TLorentzVector()
        lepton1.SetPtEtaPhiE(pT[1], eta[1], phi[1], nrg[1])


        # Calculate the parent particle four-vector
        # This is just the sum of the two lepton four-vectors

        dilepton = lepton0 + lepton1

        # Calculate any weights that may need to be applied (for MC)
        
        weight = 1
        if dataType == "MC":

            # If this is MC, we need to apply several weights
            # The "mcWeight" is needed to turn the generated distribution into a meaningful distribution
            #   The MC sample was generated in "slices", and all of the slices need to be combined
            #   The weight ensures that the slices have been combined to fit with the Standard Model
            # The scale factors correct for differences between data and MC

            weight *= getattr(tree, "mcWeight")
            weight *= getattr(tree, "scaleFactor_PILEUP")
            weight *= getattr(tree, "scaleFactor_MUON")
            weight *= getattr(tree, "scaleFactor_TRIGGER")

        # We now have the dilepton system and the event weight
        # Store the mass of the dilepton system with the relevant weight

        mll.Fill(dilepton.M(), weight)
    # We are done reading the tree into the histogram
    # Return the histogram

    return mll

# Specify input and output file names
dataFileName = "Data.root"
mcFileName = "MC.root"
histFileName = "output_histograms.root"

# Process data file
print("Running over data...")
# Open the data file as read-only
dataFile = ROOT.TFile(dataFileName, "READ")
# Get the tree in the file named "HASCO"
dataTree = dataFile.Get("HASCO")
# Read the tree into the histogram
dataHisto = readTreeToMllHistogram(dataTree, "data")
# Set the histogram to continue to exist after closing the file
dataHisto.SetDirectory(0)
dataFile.Close()
print("Done running over data")

# Process MC file
print("Running over MC...")
mcFile = ROOT.TFile(mcFileName, "READ")
mcTree = mcFile.Get("HASCO")
mcHisto = readTreeToMllHistogram(mcTree, "MC")
mcHisto.SetDirectory(0)
mcFile.Close()
print("Done running over MC")

# Write histograms to output file
print("Writing histograms to output file...")
# First create the output file, overwriting previous files with the same name
outHistFile = ROOT.TFile.Open(histFileName, "RECREATE")
# Make the file your current storage location
outHistFile.cd()
# Write the data histogram to the output file
dataHisto.Write()
# Write the MC histogram to the output file
mcHisto.Write()
# Close the output file
outHistFile.Close()
print("Done writing histograms to output file")


# Plotting the histograms
canvas = ROOT.TCanvas("canvas", "canvas", 800, 600)
# Set the y-axis to be logarithmic
canvas.SetLogy(True)

# Normalize MC histogram to data
# The two plots seem to have the same shape, but the y-axis is very different
# The MC needs to be normalized to the data
# The reason is that the MC cross-section has not been used in the weights
# (Normally you would do this, but here we don't know the correspondng MC cross-section)
# As such, we will normalize the MC histogram to have the same number of events as data

mcHisto.Scale(dataHisto.Integral() / mcHisto.Integral())

#removing auto statstics box at right corner
mcHisto.SetStats(0)
dataHisto.SetStats(0)

# Set styles for histograms
dataHisto.SetLineColor(ROOT.kBlack)
mcHisto.SetLineColor(ROOT.kRed)
# Set the line width to 2 for MC and data
dataHisto.SetLineWidth(2)
mcHisto.SetLineWidth(2)

# Set axis labels
mcHisto.GetYaxis().SetTitle("Number of events")
dataHisto.GetYaxis().SetTitle("Number of events")
mcHisto.GetXaxis().SetTitle("m_{ll} [MeV]")
dataHisto.GetXaxis().SetTitle("m_{ll} [MeV]")

# Draw MC histogram
mcHisto.Draw("hist")
# Draw data histogram on the same canvas
dataHisto.Draw("same pe")

# Add legend
legend = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)
legend.AddEntry(dataHisto, "Data", "pe")
legend.AddEntry(mcHisto, "MC", "l")
legend.Draw()

# Save the canvas as an image file
canvas.SaveAs("comparison_plot.png")

print("Plot saved as comparison_plot.png")


Running over data...
Done running over data
Running over MC...
Done running over MC
Writing histograms to output file...
Done writing histograms to output file
Plot saved as comparison_plot.png


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file comparison_plot.png has been created


In [6]:
import ROOT

# Function to read the tree and fill histograms
def readTreeToMllHistogram(tree, dataType):
    if dataType not in ["data", "MC"]:
        print(f"The data type should be \"data\" or \"MC\", not {dataType}")
        return None
    
    mll = ROOT.TH1D(dataType, f"m_{{ll}}, {dataType}", 150, 50.e3, 200.e3)
    mll.Sumw2()

    for entryNum in range(tree.GetEntries()):
        tree.GetEntry(entryNum)
        if getattr(tree, "lep_n") != 2:
            continue

        pT = getattr(tree, "lep_pt")
        eta = getattr(tree, "lep_eta")
        phi = getattr(tree, "lep_phi")
        nrg = getattr(tree, "lep_E")

        lepton0 = ROOT.TLorentzVector()
        lepton0.SetPtEtaPhiE(pT[0], eta[0], phi[0], nrg[0])
        lepton1 = ROOT.TLorentzVector()
        lepton1.SetPtEtaPhiE(pT[1], eta[1], phi[1], nrg[1])

        dilepton = lepton0 + lepton1

        weight = 1
        if dataType == "MC":
            weight *= getattr(tree, "mcWeight")
            weight *= getattr(tree, "scaleFactor_PILEUP")
            weight *= getattr(tree, "scaleFactor_MUON")
            weight *= getattr(tree, "scaleFactor_TRIGGER")

        mll.Fill(dilepton.M(), weight)

    return mll

# Specify input and output file names
dataFileName = "Data.root"
mcFileName = "MC.root"
histFileName = "output_histograms.root"

# Process data file
print("Running over data...")
dataFile = ROOT.TFile(dataFileName, "READ")
dataTree = dataFile.Get("HASCO")
dataHisto = readTreeToMllHistogram(dataTree, "data")
dataHisto.SetDirectory(0)
dataFile.Close()
print("Done running over data")

# Process MC file
print("Running over MC...")
mcFile = ROOT.TFile(mcFileName, "READ")
mcTree = mcFile.Get("HASCO")
mcHisto = readTreeToMllHistogram(mcTree, "MC")
mcHisto.SetDirectory(0)
mcFile.Close()
print("Done running over MC")

# Write histograms to output file
print("Writing histograms to output file...")
outHistFile = ROOT.TFile.Open(histFileName, "RECREATE")
outHistFile.cd()
dataHisto.Write()
mcHisto.Write()
outHistFile.Close()
print("Done writing histograms to output file")

# Plotting the histograms
canvas = ROOT.TCanvas("canvas", "canvas", 800, 600)
canvas.SetLogy(True)

dataHisto.SetStats(0)
mcHisto.SetStats(0)
dataHisto.SetLineColor(ROOT.kBlack)
mcHisto.SetLineColor(ROOT.kRed)
dataHisto.SetLineWidth(2)
mcHisto.SetLineWidth(2)
mcHisto.GetYaxis().SetTitle("Number of events")
dataHisto.GetYaxis().SetTitle("Number of events")
mcHisto.GetXaxis().SetTitle("m_{ll} [MeV]")
dataHisto.GetXaxis().SetTitle("m_{ll} [MeV]")

# Open the canvas for continuous printing
# This works for a few file types, most notably pdf files
# This allows you to put several plots in the same pdf file
# The key is the opening square-bracket after the desired file name

# Prepare the canvas for plotting
canvas.Print("comparison_plots.pdf[")

# Draw data histogram
dataHisto.Draw("pe")
canvas.Print("comparison_plots.pdf")

# Draw MC histogram (un-normalized)
mcHisto.Draw("hist")
canvas.Print("comparison_plots.pdf")

# Normalize MC histogram
mcHistoNorm = mcHisto.Clone("mcHistoNorm")
mcHistoNorm.Scale(dataHisto.Integral() / mcHisto.Integral())

# Draw the combined histogram with normalized MC
mcHistoNorm.Draw("hist")
dataHisto.Draw("same, pe")
canvas.Print("comparison_plots.pdf")

# Plot their ratio
ratio = dataHisto.Clone()
ratio.Divide(mcHistoNorm)
ratio.SetLineColor(ROOT.kRed)

# Draw the normal plots (not the ratio)
pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 1) # Set the y-axis of the top plot to be 
pad1.SetLogy(True)
pad1.SetBottomMargin(0) # Upper and lower pads are joined
pad1.Draw()             # Draw the upper pad in the canvas
pad1.cd()               # pad1 becomes the current pad
mcHistoNorm.SetTitle("")# pad1 becomes the current pad
mcHistoNorm.GetXaxis().SetLabelSize(0)  # Remove x-axis labels for the top pad
mcHistoNorm.GetXaxis().SetTitleSize(0)  # Remove x-axis title for the top pad
mcHistoNorm.GetYaxis().SetTitleSize(0.05) #Increase y-axis title size (pad is not full page)
mcHistoNorm.Draw("hist")
dataHisto.Draw("pe,same") # Draw the data points on top of the MC histo

legend = ROOT.TLegend(0.7, 0.6, 0.85, 0.75)  
legend.AddEntry(mcHistoNorm, "MC (normalized)")
legend.AddEntry(dataHisto, "Data")
legend.SetLineWidth(0) 
legend.Draw("same")

latex = ROOT.TLatex()
latex.SetNDC()
latex.SetTextSize(0.06)
latex.DrawText(0.7, 0.83, "HASCO 2018")
latex.SetTextSize(0.04)
latex.DrawText(0.7, 0.77, "Di-muon events")

# Now draw the ratio
canvas.cd()  # Go back to the main canvas before defining pad2
pad2 = ROOT.TPad("pad2", "pad2", 0, 0.05, 1, 0.3)
pad2.SetTopMargin(0)       # Upper and lower pads are joined
pad2.SetBottomMargin(0.25) # Expand the bottom margin for extra label space
pad2.Draw()                # Draw the lower pad in the canvas
pad2.cd()                  # pad2 becomes the current pad
ratio.SetTitle("")         # Turn off the title to avoid overlap
ratio.GetXaxis().SetLabelSize(0.12) # Larger x-axis labels (pad is not full page)
ratio.GetXaxis().SetTitleSize(0.12)
ratio.GetYaxis().SetLabelSize(0.1)
ratio.GetYaxis().SetTitleSize(0.15)
ratio.GetYaxis().SetTitle("Data/MC") # Change the y-axis title (this is the ratio)
ratio.GetYaxis().SetTitleOffset(0.3) # Reduce the y-axis title spacing
ratio.GetYaxis().SetRangeUser(0.5, 1.5) # Set the y-axis ratio range from 0.5 to 1.5
ratio.GetYaxis().SetNdivisions(207) # Change the y-axis tick-marks to work better
ratio.Draw("pe")    # Draw the ratio in the current pad


line = ROOT.TLine(50.e3, 1, 200.e3, 1)
line.SetLineColor(ROOT.kBlack)
line.SetLineWidth(2)
line.Draw("same")

canvas.Print("comparison_plots.pdf")

# Close the output PDF
canvas.Print("comparison_plots.pdf]")

print("Plots saved as comparison_plots.pdf")


Running over data...
Done running over data
Running over MC...
Done running over MC
Writing histograms to output file...
Done writing histograms to output file
Plots saved as comparison_plots.pdf


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file comparison_plots.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file comparison_plots.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file comparison_plots.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file comparison_plots.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file comparison_plots.pdf
Info in <TCanvas::Print>: pdf file comparison_plots.pdf has been closed


 Now you can see that there is generally agreement for di-lepton masses above roughly 70 GeV $(70 × 10^3 MeV)$
within the statistical precision. However, there is a significant difference at lower mass values. This is because
the MC sample we have used is actually only for one process, `Z → l l`, and is thus ignoring the Standard Model
`γ → l l` process. That is why data is above MC, namely we are neglecting a real process present in data when
calculating our MC expectation

---
We saw  that there is a sizable peak in the di-lepton mass spectrum, corresponding to the Z boson. If we
wanted to quantify the properties of this peak, how would we do so? One way is by trying to fit a well-defined function
to the peak in order to extract important parameters, such as its central value or width. This is important for far more
applications than just fitting resonance peaks, but resonances are one clear example.


# fitting

Fitting Gaussians in ROOT is remarkably easy, and this is a very common task.

+ The Gaussian function is pre-defined in ROOT. Create a TF1 (T for ROOT, F for function, 1 for 1-dimensional)
as below. The first argument is an arbitrary name that we need to give it, the second is the type of function
where we are using the pre-defined “gaus” for Gaussian, the third is the lower boundary to use for the fit, and
the fourth is the upper boundary to use for the fit.

In [29]:
plotFileName="fitting.pdf"
# Basic style
dataHisto.SetStats(0) # Turn off the statistics box
dataHisto.SetLineColor(ROOT.kBlack) # Set the line color to black for data
dataHisto.SetLineWidth(2) # Set the line width to 2 for data
dataHisto.GetYaxis().SetTitle("Number of events") # Set y-axis label
dataHisto.GetXaxis().SetTitle("m_{ll} [MeV]") # Set x-axis label

# Prepare the canvas for plotting
canvas = ROOT.TCanvas("canvas") # Make a canvas
canvas.cd() # Move into the canvas (so anything drawn is part of this canvas)
canvas.SetLogy(True) # Set the y-axis to be logarithmic

# Open the canvas for continuous printing
canvas.Print(plotFileName + "[") # Open for multiple plots in a single pdf

# Draw the data histogram, as data Points with Errors
dataHisto.Draw("pe")
canvas.Print(plotFileName) # Write the data histogram plot to the output file

# Fit a Gaussian to the histogram
gaussFit = ROOT.TF1("gaussfit", "gaus", 81.e3, 101.e3) # Gaussian fit in the Z-peak range
dataHisto.Fit(gaussFit, "E") # Fit the function to the histogram using improved error treatment
dataHisto.Draw("pe")

# Add the fit results to the plot
latex = ROOT.TLatex() # Create the TLatex object
latex.SetNDC() # Set coordinates to be percent-based
latex.SetTextSize(0.03)
chi2 = gaussFit.GetChisquare()
ndof = gaussFit.GetNDF()
latex.DrawText(0.5, 0.80, "Mean = %.1f GeV" % (gaussFit.GetParameter(1) / 1000))
latex.DrawText(0.5, 0.75, "Width = %.1f GeV" % (gaussFit.GetParameter(2) / 1000))
latex.DrawText(0.5, 0.70, "chi2/ndof = %.1f/%d = %.1f" % (chi2, ndof, chi2 / ndof))
canvas.Print(plotFileName) # Write the plot to the output file

# Fit a relativistic Breit-Wigner to the Z peak
k_num = "2*sqrt(2)*[0]*[1]*sqrt([0]*[0]*([0]*[0]+[1]*[1]))"
k_denom = "3.14159*sqrt([0]*[0] + sqrt([0]*[0]*([0]*[0]+[1]*[1])))"
denom = "((x*x-[0]*[0])*(x*x-[0]*[0]) + [0]*[0]*[1]*[1])"
bwFit = ROOT.TF1("bwfit", "[2]*(%s/%s)/%s" % (k_num, k_denom, denom), 50.e3, 200.e3)
bwFit.SetParameter(0, 91.1876e3) # Mass of Z boson in MeV
bwFit.SetParameter(1, 2.4952e3) # Width of Z boson in MeV
dataHisto.Fit(bwFit, "E")
dataHisto.Draw("pe")
chi2 = bwFit.GetChisquare()
ndof = bwFit.GetNDF()
latex.DrawText(0.5, 0.80, "Mean = %.1f GeV" % (bwFit.GetParameter(0) / 1000))
latex.DrawText(0.5, 0.75, "Width = %.1f GeV" % (bwFit.GetParameter(1) / 1000))
latex.DrawText(0.5, 0.70, "chi2/ndof = %.1f/%d = %.1f" % (chi2, ndof, chi2 / ndof))
canvas.Print(plotFileName)

# Calculate the ratio
ratio = dataHisto.Clone()
ratio.Divide(bwFit)
ratio.SetLineColor(ROOT.kRed)

# Draw the normal plots (not the ratio)
pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 1)
pad1.SetLogy(True) # Set the y-axis of the top plot to be logarithmic
pad1.SetBottomMargin(0) # Upper and lower pads are joined
pad1.Draw() # Draw the upper pad in the canvas
pad1.cd() # pad1 becomes the current pad
dataHisto.SetTitle("") # Remove the plot title
dataHisto.GetXaxis().SetLabelSize(0) # Remove x-axis labels for the top pad
dataHisto.GetXaxis().SetTitleSize(0) # Remove x-axis title for the top pad
dataHisto.GetYaxis().SetTitleSize(0.05) # Increase y-axis title size (pad is not full page)
dataHisto.Draw("pe,same") # Draw the data points on top of the MC histo

# Add a legend to the top pad
legend = ROOT.TLegend(0.7, 0.6, 0.85, 0.75)
legend.AddEntry(dataHisto, "Data")
legend.AddEntry(bwFit, "Breit-Wigner Fit")
legend.SetLineWidth(0)
legend.Draw("same")

# Add other labels with TLatex
latex.SetTextSize(0.06)
latex.DrawText(0.7, 0.83, "HASCO 2018")
latex.SetTextSize(0.04)
latex.DrawText(0.7, 0.77, "Di-muon events")

# Draw the mass, width, and chi2/ndof
latex.DrawText(0.4, 0.80, "Mean = %.1f GeV" % (bwFit.GetParameter(0) / 1000))
latex.DrawText(0.4, 0.75, "Width = %.1f GeV" % (bwFit.GetParameter(1) / 1000))
latex.DrawText(0.4, 0.70, "chi2/ndof = %.1f/%d = %.1f" % (chi2, ndof, chi2 / ndof))

# Now draw the ratio
canvas.cd() # Go back to the main canvas before defining pad2
pad2 = ROOT.TPad("pad2", "pad2", 0, 0.05, 1, 0.3)
pad2.SetTopMargin(0) # Upper and lower pads are joined
pad2.SetBottomMargin(0.25) # Expand the bottom margin for extra label space
pad2.Draw() # Draw the lower pad in the canvas
pad2.cd() # pad2 becomes the current pad
ratio.SetTitle("") # Turn off the title to avoid overlap
ratio.GetXaxis().SetLabelSize(0.12) # Larger x-axis labels (pad is not full page)
ratio.GetXaxis().SetTitleSize(0.12) # Larger x-axis title (pad is not full page)
ratio.GetYaxis().SetLabelSize(0.1) # Larger y-axis labels (pad is not full page)
ratio.GetYaxis().SetTitleSize(0.15) # Larger y-axis title (pad is not full page)
ratio.GetYaxis().SetTitle("Data/Fit") # Change the y-axis title (this is the ratio)
ratio.GetYaxis().SetTitleOffset(0.3) # Reduce the y-axis title spacing
ratio.GetYaxis().SetRangeUser(0.5, 1.5) # Set the y-axis ratio range from 0.5 to 1.5
ratio.GetYaxis().SetNdivisions(207) # Change the y-axis tick-marks to work better
ratio.SetMarkerStyle(3) # Change the marker style to stars
ratio.SetMarkerColor(ROOT.kRed) # Change the marker colour to red
ratio.Draw("pe") # Draw the ratio in the current pad, without errors

# Add a line at 1 to the ratio plot
line = ROOT.TLine(50.e3, 1, 200.e3, 1) # Draw a line at 1 from 50 GeV to 200 GeV (full plot)
line.SetLineColor(ROOT.kBlack) # Set the line colour to black
line.SetLineWidth(2) # Set the line width to 2
line.Draw("same") # Draw the line on the same plot as the ratio

# Write the ratio plot to the output plot file
canvas.Print(plotFileName)

# Close the output file
canvas.Print(plotFileName + "]")


 FCN=10889 FROM MINOS     STATUS=SUCCESSFUL     20 CALLS         225 TOTAL
                     EDM=6.81397e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     6.26913e+03   4.30927e+01   5.26908e-02  -2.92835e-06
   2  Mean         9.06248e+04   1.47209e+01  -1.18182e-02  -7.20941e-07
   3  Sigma        3.29138e+03   1.73890e+01   1.73890e+01  -4.18851e-02
 FCN=1841.93 FROM MINOS     STATUS=SUCCESSFUL     20 CALLS         264 TOTAL
                     EDM=3.56586e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           9.06734e+04   1.47171e+01   3.92787e-03  -5.33917e-07
   2  p1           4.82007e+03   2.64479e+01  -3.86546e-03   1.03413e-07
   3  p2           3.78873e-03   1.

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file fitting.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file fitting.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file fitting.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file fitting.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file fitting.pdf
Info in <TCanvas::Print>: pdf file fitting.pdf has been closed
